In [1]:
# Display
import pandas as pd
import geopandas as gpd
# Settings
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:,.2f}".format

# GCS, del later since this will presumbly be read from a script that cleans up the data
# GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

# My utilities
import _utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
df_statewide = pd.read_excel(f"{_utils.GCS_FILE_PATH}{FILE}", sheet_name="fake")

In [3]:
# Count of projects by district - for mapping
districts_gdf = _utils.summarize_by_project_names(df_statewide, "District")

In [4]:
# Add official Caltrans District names
district_dictionary = {
    7: "07 - Los Angeles",
    4: "04 - Oakland",
    2: "02 - Redding",
    9: "09 - Bishop",
    10: "10 - Stockton",
    11: "11 - San Diego",
    3: "03 - Marysville",
    12: "12 - Irvine",
    8: "08 - San Bernardino",
    5: "05 - San Luis Obispo",
    6: "06 - Fresno",
    1: "01 - Eureka",
    75: "75 - HQ",
    74: "74 - HQ",
    0: "None",
}

In [5]:
# Reverse a dictionary
# https://stackoverflow.com/questions/483666/reverse-invert-a-dictionary-mapping
inv_district_map = {v: k for k, v in district_dictionary.items()}

In [6]:
# Remap districts to just the number
# 1st create duplicate column
districts_gdf['district_number'] = districts_gdf['District']

In [7]:
# Map the inverse
districts_gdf['district_number'] = districts_gdf['district_number'].replace(
    inv_district_map
)

In [8]:
# Join gdf above with Caltrans shape
districts_gdf = _utils.create_caltrans_map(districts_gdf)

In [9]:
# Drop cols
districts_gdf = districts_gdf.drop(columns = ['DISTRICT','OBJECTID','Region', 'district_number'])

In [11]:
_utils.geojson_gcs_export(
    districts_gdf,
    _utils.GCS_FILE_PATH,
    "districts",
)